In [1]:
#    This is a test
from __future__ import division
corpus = 'data/delorme.com_shu.pages_89.txt'


import math
import os
import sys

from config import CONFIG

from tests import tests1, tests2

from normalizer import Normalizer
from lib.CharacterIndex import CharacterIndex
from lib.NaiveTokenizer import NaiveTokenizer
from lib.TextStreamer import TextStreamer

from lib.Tools import (
    FreqDist,
    tokenizer,
    splitter
)

from collections import (
    Counter,
    defaultdict as deft
)



def freq_ratio(maxim, minim, log_base=2.0):
    if maxim < 10:
        return False
    multiplier = math.log(maxim, log_base)
#     print maxim, minim, maxim / minim, multiplier, (multiplier * multiplier)
    if maxim / minim >= (multiplier * multiplier):
        return True
    else:
        return False


def get_name(template):
    i = 1
    while True:
        name = template % i
        if not os.path.exists(name):
            return name
        i += 1

for C in CONFIG:
    
    tests = tests1.items() + tests2.items()

    #	Collect input from large text file:
    streamer = TextStreamer(corpus)

    dump = []
    for doc in streamer:
        for sent in splitter(doc):
            dump += tokenizer(sent)
    freq_dist = Counter(dump)
    

    #	Map all character n-grams to words, and all words to their
    #	character n-grams
    index = CharacterIndex(dump + tests, min_r=C['sim_thres'])
    index.build()

    
    hits = 0
    empty = 0


    template = 'logs/test.%d.txt'
    file = open(get_name(template), 'wb')
    file.write('\n'.join(['%s=%s' % (str(x), str(y)) for x, y in C.items()]) + '\n')

    for correct, error in tests:
        similars = index[error]
        if not similars:
            empty += 1
        f_w = freq_dist[error]
        if not similars:
            continue
        similars.sort(
            key=lambda x: freq_dist[x[0]],
            reverse=True
        )
        if similars[0][0] == correct:
            hits += 1
    #        print error, '>', correct, hits, hits / float(len(tests) - empty)
        else:
            file.write('error="%s"  hypothesis="%s"  human="%s"\n' % (error, similars[0][0], correct))
    file.write('hits = "%s"  precision = "%s" recall = "%s" \n' % (hits, hits / float(len(tests) - empty), hits / (hits + empty)))

    file.close()
    
    
    print C, ('hits = "%s"  precision = "%s" recall = "%s" \n' % (hits, hits / float(len(tests) - empty), hits / (hits + empty)))


{'min_Count': 10, 'nb_sent': 1000, 'freq_ratio': 10, 'chgram': 2, 'sim_thres': 0.9} hits = "250"  precision = "0.919117647059" recall = "0.51867219917" 

{'min_Count': 10, 'nb_sent': 1000, 'freq_ratio': 10, 'chgram': 2, 'sim_thres': 0.88} hits = "286"  precision = "0.916666666667" recall = "0.598326359833" 

{'min_Count': 10, 'nb_sent': 10000, 'freq_ratio': 10, 'chgram': 2, 'sim_thres': 0.85} hits = "325"  precision = "0.905292479109" recall = "0.691489361702" 

{'min_Count': 10, 'nb_sent': 10000, 'freq_ratio': 10, 'chgram': 2, 'sim_thres': 0.85} hits = "325"  precision = "0.905292479109" recall = "0.691489361702" 

{'min_Count': 10, 'nb_sent': 20000, 'freq_ratio': 10, 'chgram': 2, 'sim_thres': 0.9} hits = "250"  precision = "0.919117647059" recall = "0.51867219917" 

